<a href="https://colab.research.google.com/github/marathomas/meerkat/blob/master/05_Comparison_to_traditional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embedding in traditional features

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import sys
os.system('pip install "librosa==0.7.1"')
import librosa
import numpy as np
import statistics
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.legend import Legend
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc, gridspec
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import umap
from scipy.stats import zscore
from sklearn import metrics
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.legend import Legend
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc, gridspec
import matplotlib
import seaborn as sns
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
import scipy
from scipy.spatial.distance import euclidean as euclid
from scipy.spatial import distance_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


PROJECT_PATH = "/content/drive/My Drive/meerkat/" 
FIGURES = PROJECT_PATH+"figures/"
DATA = PROJECT_PATH+"data/"

os.chdir(PROJECT_PATH)

# Load datasets

In [6]:
df_name = '2020-09-20_embedding_denoised_melspectro_focal_focal'
df = pd.read_pickle(DATA+'df_denoised_balanced')
df = df[df.nonfocal_yn == 'n'] # for focal
df.reset_index(inplace=True)

In [4]:
features = pd.read_csv(PROJECT_PATH+"data/frants/mara_features_allSNR.csv")

In [5]:
features.head(3)

,CALL_INDEX,Call_Type,Energy_Duration,Spectral_Entropy,Temporal_Entropy,Cepstral_Peak_Prominence,Centroid_Frequency,Peak_Frequency,lower_frequency_10dB,Upper_Frequency_10dB,Bandwidth_RMS,Bandwidth_10dB,F0_mean,F0_start,F0_mid,F0_end,SNR
0,0,sn,0.034750,4.824401,0.856071,22.450264,1380.572976,1296.8750,1195.3125,1398.4375,399.863732,203.125,1290.178571,1296.875,1296.8750,1273.4375,8.456472
1,1,al,0.028500,5.877919,0.848490,20.901762,1218.884450,1289.0625,312.5000,1437.5000,562.881075,1125.000,1308.593750,1359.375,1335.9375,1265.6250,-3.195684
2,2,al,0.039375,4.822398,0.915894,19.453461,1407.414980,1203.1250,1078.1250,1312.5000,584.475114,234.375,1208.007812,1171.875,1203.1250,1242.1875,17.318290


Check for NA vals

In [8]:
features.isnull().sum()

CALL_INDEX                  0
Call_Type                   0
Energy_Duration             0
Spectral_Entropy            0
Temporal_Entropy            0
Cepstral_Peak_Prominence    0
Centroid_Frequency          0
Peak_Frequency              0
lower_frequency_10dB        0
Upper_Frequency_10dB        0
Bandwidth_RMS               0
Bandwidth_10dB              0
F0_mean                     0
F0_start                    0
F0_mid                      0
F0_end                      0
SNR                         0
dtype: int64

# Dimensionality reduction

In [23]:
embedding_dict = {}

def plotly_viz(method):

  embedding = embedding_dict[method]
  x, y, z = embedding[:,0], embedding[:,1], embedding[:,2] 
  scat_labels = feat_labels
  labeltypes = sorted(list(set(scat_labels)))

  # makes colors same as AVGN visualization
  pal = sns.color_palette("Set2", n_colors=len(labeltypes))
  color_dict = dict(zip(labeltypes, pal))
  c = [color_dict[val] for val in scat_labels]

  fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                    mode='markers',
                                    hovertext = feat_labels,
                                    marker=dict(
                                        size=4,
                                        color=c,                # set color to an array/list of desired values
                                        opacity=0.8
                                        ))])

  fig.update_layout(scene = dict(
                      xaxis_title=method+'1',
                      yaxis_title=method+'2',
                      zaxis_title=method+'3'),
                      width=700,
                      margin=dict(r=20, b=10, l=10, t=10))

  return fig

plotly_viz("UMAP").show()

## Prepare input data

Choose data

In [ ]:
# A) Full dataset
data = features.drop(columns=['CALL_INDEX', 'Call_Type', 'SNR'])
labels = features.Call_Type

In [43]:
# B) reduce set to only high quality (signal to noise > 10 dB)
data = features.loc[features.SNR>10,]
labels = data.Call_Type
data = data.drop(columns=['CALL_INDEX', 'Call_Type', 'SNR'])
data.shape

(5306, 14)

In [44]:
data.head(3)

,Energy_Duration,Spectral_Entropy,Temporal_Entropy,Cepstral_Peak_Prominence,Centroid_Frequency,Peak_Frequency,lower_frequency_10dB,Upper_Frequency_10dB,Bandwidth_RMS,Bandwidth_10dB,F0_mean,F0_start,F0_mid,F0_end
2,0.039375,4.822398,0.915894,19.453461,1407.414980,1203.125,1078.1250,1312.5000,584.475114,234.3750,1208.007812,1171.8750,1203.125,1242.187500
3,0.032625,3.931041,0.793763,21.235393,1312.126084,1296.875,1195.3125,1414.0625,161.830896,218.7500,1295.572917,1304.6875,1296.875,1281.250000
4,0.041625,4.610101,0.953889,16.560427,864.562632,718.750,593.7500,1476.5625,347.535126,882.8125,719.238281,710.9375,718.750,724.609375


z-transform

In [45]:
data_z = StandardScaler().fit_transform(data)

## UMAP

In [46]:
#UMAP
reducer = umap.UMAP(low_memory=True, n_components = 10, min_dist=0)
embedding_dict['UMAP'] =  reducer.fit_transform(data_z)

In [47]:
plotly_viz("UMAP").show()

## tSNE

In [ ]:
model = TSNE(n_components=3, init='pca', random_state=0, perplexity=data.shape[0]/100, learning_rate=round(data.shape[0]/12))
embedding_dict['tSNE'] = model.fit_transform(data_z) 

plotly_viz('tSNE').show()

## PCA

In [38]:
p = PCA(n_components=10, random_state=728)
p.fit(np.transpose(data))
embedding_dict['PCA'] = np.transpose(p.components_)

plotly_viz("PCA").show()

# Visualization

In [32]:
def plotly_viz(method):

  embedding = embedding_dict[method]
  x, y, z = embedding[:,0], embedding[:,1], embedding[:,2] 
  scat_labels = labels
  labeltypes = sorted(list(set(scat_labels)))

  # makes colors same as AVGN visualization
  pal = sns.color_palette("Set2", n_colors=len(labeltypes))
  color_dict = dict(zip(labeltypes, pal))
  c = [color_dict[val] for val in scat_labels]

  fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                    mode='markers',
                                    hovertext = labels,
                                    marker=dict(
                                        size=4,
                                        color=c,                # set color to an array/list of desired values
                                        opacity=0.8
                                        ))])

  fig.update_layout(scene = dict(
                      xaxis_title=method+'1',
                      yaxis_title=method+'2',
                      zaxis_title=method+'3'),
                      width=700,
                      margin=dict(r=20, b=10, l=10, t=10))

  return fig

plotly_viz("UMAP").show()